# Data Wrangling with MongoDB

Author: Hana AlMashari

Date: June 3, 2017

Open Street Map: Manchester, England, United Kingdom

In this project I will show the process of audting by identifying the problems in the data ,then cleaning some of this problems,finally will write the data to JSON file and import it to MongoDB and run some queries.


### Why Manchester : 

I’ve chosen (manchester_england) Data set since I'm planning to travel to it in my next vacation.So I found it good idea to explore the places and street and many things through this project.

reference:(https://mapzen.com/data/metro-extracts/metro/manchester_england/101913963/Manchester/)



## Problems Encountered in the Data:

#### Problems in phone numbers attribut:

- Phone number written differently with spaces.
- There's Special characters and letters.

        <tag k="phone" v="+441612366222"/>
        <tag k="phone" v="0161 839 9035"/>
        <tag k="phone" v="01606 46666"/>

#### Problems in street attribute ( addr:street ):
- Some street names has special characters.
- Street types (rd , road should be writtend in the same way)
- Street type like Str in the beggining of the street name instead of the end like the blew: 
		<tag k="addr:street" v="St Paul’s Place"/>
    	<tag k="addr:street" v="St Mary&#39;s Road"/>
        
#### Problems in postcode attribut (addr:postcode):
When the city is Manchester the post code has the prefix M , and the belows post code are out of the range:

		<tag k="addr:postcode" v="SK23 7LP"/>
		<tag k="addr:postcode" v="SK9 3PA"/>
		<tag k="addr:postcode" v="SK9 5LR"/>
		<tag k="addr:postcode" v="SK9 1PZ"/>
		<tag k="addr:postcode" v="SK9 7JT"/>
  
#### General problem :
- There're data not in city Manchester
        <tag k="addr:city" v="Manchester"/>

In [291]:
#Importing 
import sys
import pprint
print(sys.version)#python version 
print(sys.version_info)
#---------------------------------
import xml.etree.ElementTree as ET #ET.iterparse(filename):
from collections import defaultdict #using defaultdict(int)
import re
import os
import xml.etree.cElementTree as ET
import codecs
import json
from pymongo import MongoClient
import os
import math

2.7.13 |Anaconda custom (x86_64)| (default, Dec 20 2016, 23:05:08) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
sys.version_info(major=2, minor=7, micro=13, releaselevel='final', serial=0)


In [292]:
#Upload the file
OSM_FILE = "/Users/hana/Desktop/datascience/manchester_england.osm" 

## Data Overview:

Let's take an overviw of the file containing the data we're going to wrangle.In the below output is the size of the data which seems quit big one.

In [293]:
#Return the size, in bytes,
os.path.getsize('/Users/hana/Desktop/datascience/manchester_england.osm')

357820941

When Changing the unit form bytes to MB is about 357.8 MB

In order to Audit the DataSet we need to know overview of the Data by counting the number of unique element types in the OSM_FILE.Bacicaly we counts number of tags in the osm_file.

In [294]:

# %%writefile mapparser.py
#!/usr/bin/env python

import xml.etree.ElementTree as ET
import pprint

#count the tags in a given file name 
def count_tags(filename):
    tags = {}
    for ev,elem in ET.iterparse(filename):
        tag = elem.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag]+=1
    return tags

def test():

    tags = count_tags(OSM_FILE)
    pprint.pprint(tags)

if __name__ == "__main__":
    test()


{'bounds': 1,
 'member': 31102,
 'nd': 1981674,
 'node': 1603430,
 'osm': 1,
 'relation': 2426,
 'tag': 875731,
 'way': 224173}


## Auditing the Data:
In auditing we need to 
- List the issues and 
- List expected value 

In the below functions we go into tags and see the counts of the following cases:
- "lower":Tags that contain only lowercase letters and are valid.
- "lower_colon": Valid tags with a colon in their names.
- "problemchars": Tags with problematic characters.

In [295]:
#Before you process the data and add it into your database, we have to check the
#"k" value for each "<tag>" and see if there are any potential problems.
#Here I have built three regular expressions to check for certain patterns
#in the tags.
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
        
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map(OSM_FILE)
pprint.pprint(keys)

{'lower': 651974, 'lower_colon': 101387, 'other': 122367, 'problemchars': 3}


Now, let's see the number of users contributed to the "manchester_england" map

In [296]:
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])

    return users

users = process_map(OSM_FILE)
len(users)

2155

I will be focusing on auditing and cleaning the following attributes:


1. Street names
2. Phonenumbers

In cleaning the street names ,we looped through the Street names and by going through the values i have maintan the expected list values of street types.

In [297]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road","Cross","Village","Fold",
            "Trail", "Parkway", "Commons" ,"West" , "East" , "Way" , 'South','Estate' , "Croft" , "Gate","Walk","North","Terrace",
            'Walk' ,'Hill','Grove', 'Mews','Gardens','Park','Centre','Brow','Chase','Arcade','Close','Crescent' ,'Green',
           "Parade" ,"Quay","Grange","House","Knowle","View" ,"Mall" ,"Quays" ,"Meadow" ,"End"]

mapping = { "St": "Street",
            "St.": "Street",
            "street": "Street",
            "Ave": "Avenue",
            "avenue" : "Avenue",
            "Rd." : "Road",
            "road": "Road",
            "Rd": "Road",
            "Gates":"Gate",
            "square" :"Square",
            "east":"East",
            "Meadows" : "Meadow",
            "Cottages":"Cottage",
            "Ends":"End",
            "Orchards": "Orchard",
            "Heyes": "Hey"
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types


st_types = audit(OSM_FILE)

In auditing the street types.

* Case#1 : I added the street types that's in the file into the expected list.
* Case#2 : Unify the types written in different types like the below: 
For the road , Rd i'm going to unify it to be Road  
For avenue, Ave ==> Avenue 
Square , sq ==> Square 
* Case#3 :

ّFor the street type with 's' at the end I unify the type to be single 
like below 
             'Meadow': {'Round Meadow'},
             'Meadows': {'High Meadows'},
so all will be "Meadow" 

In [328]:
def update_name(name, mapping):
     sorted_keys = sorted(mapping.keys(), key=len, reverse=True)
     for abbrv in sorted_keys:
         if(abbrv in name):
             name.rstrip() #removing spaces at the end of the street type
             return name.replace(abbrv, mapping[abbrv])
     return name


def Run():
    count = 0
    for st_type, ways in st_types.iteritems():
        count +=1
        if count ==10:
            break
        for name in ways:
            better_name = update_name(name, mapping)
            print name,"=>", better_name

Run()

Pink => Pink
Guywood Cottages => Guywood Cottage
Rookery Cottages => Rookery Cottage
Wharf Cottages => Wharf Cottage
Shudehill => Shudehill
Kingsway => Kingsway
Burton Rd => Burton Road
Mosley Rd => Mosley Road
Stockport Rd => Streetockport Rd
Wilmslow Rd => Wilmslow Road
Churchfields => Churchfields
Deangate => Deangate
Dolefield => Dolefield
Alderley Lodge => Alderley Lodge


Let's see how different patteren phone number wrriten in dataset


In [299]:
def is_phone(elem):
    return (elem.attrib['k'] == "phone")

def is_city(elem): # needed if i will chek manchester
    return (elem.attrib['k'] == "addr:city")
def audit(osmfile):
    osm_file = open(osmfile, "r")
    phone_types = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone(tag):
                    phone_types.append(tag.attrib['v'])

    return phone_types

phones = audit(OSM_FILE)

After checking the phone number i have notice that there are spaces and special characters in the phone number.
Let's update the phone number.

In [300]:
def update_phone (phonenumber):
    phone_r = phonenumber.strip()
    phone_s = phone_r.replace(" ", "")
    if phone_s[0] == "+":
        phone_s = phone_s[2:]
        phone_s = re.sub('[^a-zA-Z0-9-_*.]', '', phone_s) #replace special character with "" to remove it
    return phone_s
    
def Run():
    for phone in phones:
        ph = update_phone(phone)
        print phone ,"=>",ph


## Preparing for MongoDB 
In order to prepare for MongoDB we need first to convert the file To JSON format keeping the original file without modification.
The structure of the data would be like this:

Form this format:

    <node id="27169437" lat="53.3930029" lon="-2.2350969" version="3" timestamp="2017-01-20T23:17:54Z" changeset="45341398" uid="334129" user="murble">
		<tag k="addr:city" v="Stockport"/>
		<tag k="addr:housenumber" v="144"/>
		<tag k="addr:postcode" v="SK8 4AB"/>
		<tag k="addr:street" v="Gatley Road"/>
		<tag k="amenity" v="pub"/>
		<tag k="fhrs:id" v="144880"/>
		<tag k="name" v="The Horse and Farrier"/>
		<tag k="smoking" v="outside"/>
	</node>
---------------------------------------------------------------------------------------------------------

To this 


{
    "id": "27169437",
    "type: "node",
    "visible":"true",
    "created": {
                  "version":"3",
                  "changeset":"45341398",
                  "timestamp":"2017-01-20T23:17:54Z",
                  "user":"murble",
                  "uid":"334129"
               },
    "pos": [53.3930029, -2.2350969],
    "address": {  
                  "housenumber": "144",
                  "postcode": "SK8 4AB",
                  "street": "Gatley Road"
                  "city"  : "Stockport"
               },
    "amenity": "pub",
    "name": "The Horse and Farrier",
    "phone": ""
}

The transform will follow these rules in josn:
* Process only 2 types of top level tags: node and way
* All attributes of node and way should be turned into regular key/value pairs.
* (created key) has  version, changeset, timestamp, user, uid
* latitude and longitude attributes should be added to a pos array
* Ignore second level tag "k" value that contains problemtic characters.
* scond level tag that starts with addr: will be added to address dic
* any tag contains ":" will deal with it like any other tag.

In [301]:
#prosses to josn file 
CREATED = ["version", "changeset", "timestamp", "user", "uid"]

def shape_element(element):
    node = {}    
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        
        # Parse attributes josn
        for attrib in element.attrib:

            # Creation Data
            if attrib in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                if attrib == 'timestamp':
                    node['created'][attrib] = datetime.strptime(element.attrib[attrib], '%Y-%m-%dT%H:%M:%SZ')
                else:
                    node['created'][attrib] = element.get(attrib)

            # Parse location (Latitude and Longitude )
            if attrib in ['lat', 'lon']:
                lat = float(element.attrib.get('lat'))
                lon = float(element.attrib.get('lon'))
                node['pos'] = [lat, lon]

            # Parse the rest of attributes
            else:
                node[attrib] = element.attrib.get(attrib)
            
        # Process tags
        for tag in element.iter('tag'):
            key   = tag.attrib['k']
            value = tag.attrib['v']
            if not problemchars.search(key):
                #applying the phone cleaning
                if key == "phone":
                    updated_phone = update_phone(tag.attrib['v'])
                    node['phone'] = updated_phone
                # addr Tags with :
                if lower_colon.search(key) and key.find('addr') == 0:
                    if 'address' not in node:
                        node['address'] = {}
                    sub_attr = key.split(':')[1]
                    # Apply the cleaning of the addr:street 
                    if is_street_name(tag):
                        updated_name = update_name(tag.attrib['v'],mapping)
                        node['address'][sub_attr] = updated_name
                    else:    
                        node['address'][sub_attr] = value
                # All other tags that don't begin with "addr"
                elif not key.find('addr') == 0:
                    if key not in node:
                        node[key] = value     
                else:
                    node["tag:" + key] = value

        # Process nodes
        for nd in element.iter('nd'):
            if 'node_refs' not in node:
                node['node_refs'] = []
            node['node_refs'].append(nd.attrib['ref'])

        return node
    else:
        return None

In [302]:
import json
from bson import json_util

def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)    
    with open(file_out, "wb") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                if pretty:
                    fo.write(json.dumps(el, indent=2, default=json_util.default)+"\n")
                else:
                    fo.write(json.dumps(el, default=json_util.default) + "\n")

json_file = process_map(OSM_FILE)

In [288]:
import json
from bson import json_util

def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)    
    with open(file_out, "wb") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                if pretty:
                    fo.write(json.dumps(el, indent=2, default=json_util.default)+"\n")
                else:
                    fo.write(json.dumps(el, default=json_util.default) + "\n")

json_file = process_map(OSM_FILE)

## MangoDB  
I'm going to import the .json which is the cleand data into mangoDB 

In [303]:
db_name = "manchester_england"

In [304]:
# method to get database by its name: 
def get_db(db_name):
    # For local use
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

In [305]:
import signal
import subprocess

pro = subprocess.Popen('mongod', preexec_fn = os.setsid)

In [306]:
collection = 'manchester_england_data'
working_directory = '/Users/hana/Desktop/datascience/'
#json_file = filename + '.json'

mongoimport_cmd = 'mongoimport -h 127.0.0.1:27017 ' + \
                  '--db ' + db_name + \
                  ' --collection ' + collection + \
                  ' --file ' + working_directory +"manchester_england.osm.json"

# Execute the command
print 'Executing: ' + mongoimport_cmd
subprocess.call(mongoimport_cmd.split())

Executing: mongoimport -h 127.0.0.1:27017 --db manchester_england --collection manchester_england_data --file /Users/hana/Desktop/datascience/manchester_england.osm.json


0

In [307]:
print collection

manchester_england_data


In [308]:
# info about the data: 
if __name__ == "__main__":
    db = get_db(db_name)
    print "Database Name: " + db.name
    
    #print "Sample Data Point: "
    pprint.pprint (db.manchester_england_data.find_one())

Database Name: manchester_england
{u'_id': ObjectId('592ca2a2230fad747dc97875'),
 u'changeset': u'10388479',
 u'created': {u'changeset': u'10388479',
              u'timestamp': datetime.datetime(2012, 1, 14, 14, 58, 6),
              u'uid': u'4559',
              u'user': u'RichardB',
              u'version': u'2'},
 u'id': u'219434',
 u'pos': [53.3342527, -2.575179],
 u'timestamp': u'2012-01-14T14:58:06Z',
 u'type': u'node',
 u'uid': u'4559',
 u'user': u'RichardB',
 u'version': u'2'}


After getting mangodb database done , here we can get some info about it
#### Number of ways and nodes

In [309]:
print db.manchester_england_data.find({"type": "node"}).count()
print db.manchester_england_data.find({"type": "way"}).count()


4810290
672519


#### Number of Unique Users

In [310]:
print len(db.manchester_england_data.distinct("created.uid"))

2133


In [311]:
def queries ():  
    
    print "Top Three Contributors: "
    top_users = db.manchester_england_data.aggregate([{"$group":{"_id":"$created.uid", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":3}])
    for user in top_users:
        print user
    print "======================================================================================="
    print "Top Three Amenities: "
    top_amenities = db.manchester_england_data.aggregate([{"$match":{"amenity":{"$exists":1}}}, {"$group":{"_id":"$amenity","count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":3}])
    for amenity in top_amenities:
        pprint.pprint(amenity)
        
    print "======================================================================================="
    print "Top Three Banks: "
    top_banks = db.manchester_england_data.aggregate([{'$match': {'amenity': 'bank'}},{'$group': {'_id': '$name','count': {'$sum': 1}}},{'$sort': {'count': -1}},{'$limit': 3}])
    for bank in top_banks:
        pprint.pprint(bank)
        
    print "======================================================================================="   
    print "Top Three Cities: "
    top_cities = db.manchester_england_data.aggregate([{'$match': {'address.city': {'$exists': 1}}},{'$group': {'_id': '$address.city', 'count': {'$sum': 1}}},{'$sort': {'count': -1}},{'$limit': 3}])
    for city in top_cities:
        pprint.pprint(city)

In [312]:
queries()

Top Three Contributors: 
{u'count': 746442, u'_id': u'4559'}
{u'count': 288858, u'_id': u'714383'}
{u'count': 259413, u'_id': u'101150'}
Top Three Amenities: 
{u'_id': u'parking', u'count': 12612}
{u'_id': u'pub', u'count': 4686}
{u'_id': u'school', u'count': 4056}
Top Three Banks: 
{u'_id': u'Barclays', u'count': 90}
{u'_id': u'NatWest', u'count': 78}
{u'_id': u'RBS', u'count': 72}
Top Three Cities: 
{u'_id': u'Manchester', u'count': 7671}
{u'_id': u'Romiley', u'count': 2310}
{u'_id': u'Northwich', u'count': 963}


As we can see from the queries result , The top contributor user is '4559' , Manchecter has agreat number of parking area which is agood info for me to take into considration renting a car when i go for vacation.
i have list the cites which is clear that not only manchecter city data is exsported from the site. 

## Other Ideas About the Dataset:


Since I am coffee addict, I would like to explore Manchester data Set and see the Following Cases:
- Listing all the cafes 
- List the cafe shops that have outdoor earea where smoking not allowed 
- Search for Cafe North and git the address information , one of my freiend recommend it to me 

In [313]:
#list all the cafes in the dataset
list_cafe = db.manchester_england_data.find({"amenity": "cafe"})


In [322]:
no_smoking = db.manchester_england_data.find({'$and':[{'outdoor_seating': 'yes'},{"smoking":"no"},{ 'amenity':'cafe'}]})
count=0
for c in no_smoking:
    count+=1
    if count ==3:
        break
    print c
    print "============================================================================"

{u'changeset': u'21702777', u'amenity': u'cafe', u'uid': u'51722', u'created': {u'changeset': u'21702777', u'version': u'2', u'user': u'Chris Parker', u'timestamp': datetime.datetime(2014, 4, 15, 6, 12, 16), u'uid': u'51722'}, u'timestamp': u'2014-04-15T06:12:16Z', u'wheelchair': u'yes', u'outdoor_seating': u'yes', u'cuisine': u'coffee_shop', u'pos': [53.473716, -2.240016], u'name': u'Starbucks', u'internet_access:fee': u'no', u'version': u'2', u'user': u'Chris Parker', u'takeaway': u'yes', u'smoking': u'no', u'_id': ObjectId('592ca2b0230fad747dd2a59d'), u'type': u'node', u'id': u'1502881868', u'internet_access': u'wlan'}
{u'changeset': u'23546716', u'amenity': u'cafe', u'uid': u'51722', u'created': {u'changeset': u'23546716', u'version': u'2', u'user': u'Chris Parker', u'timestamp': datetime.datetime(2014, 7, 2, 11, 38, 25), u'uid': u'51722'}, u'timestamp': u'2014-07-02T11:38:25Z', u'wheelchair': u'limited', u'outdoor_seating': u'yes', u'pos': [53.480451, -2.244931], u'version': u'2',

In [315]:
cafeshops = db.manchester_england_data.find({'$and':[{'address.street': 'Oxford Road'},{ 'amenity':'cafe'}]})
for cafe in cafeshops:
    print cafe
    print "============================================================================"

{u'changeset': u'29742034', u'amenity': u'cafe', u'uid': u'5409', u'created': {u'changeset': u'29742034', u'version': u'4', u'user': u'peter_james', u'timestamp': datetime.datetime(2015, 3, 26, 0, 19, 56), u'uid': u'5409'}, u'timestamp': u'2015-03-26T00:19:56Z', u'pos': [53.4726652, -2.2395845], u'cuisine': u'coffee_shop', u'version': u'4', u'user': u'peter_james', u'address': {u'city': u'Manchester', u'street': u'Oxford Road', u'housenumber': u'14', u'postcode': u'M1 5QA'}, u'operator': u'Whitbread Group plc', u'_id': ObjectId('592ca2ab230fad747dcf09eb'), u'type': u'node', u'id': u'796761399', u'name': u'Costa'}
{u'changeset': u'29742034', u'amenity': u'cafe', u'uid': u'5409', u'created': {u'changeset': u'29742034', u'version': u'4', u'user': u'peter_james', u'timestamp': datetime.datetime(2015, 3, 26, 0, 19, 56), u'uid': u'5409'}, u'timestamp': u'2015-03-26T00:19:56Z', u'pos': [53.4726652, -2.2395845], u'cuisine': u'coffee_shop', u'version': u'4', u'user': u'peter_james', u'address':

In [323]:
cafes = db.manchester_england_data.find({'$and':[{'name': 'Cafe North'},{ 'amenity':'cafe'}]})
count=0
for cafe in cafes:
    count+=1
    if count ==3:
        break
    print cafe
    print "============================================================================"

{u'changeset': u'7697865', u'amenity': u'cafe', u'uid': u'428524', u'created': {u'changeset': u'7697865', u'version': u'2', u'user': u'Daveysawyer', u'timestamp': datetime.datetime(2011, 3, 28, 15, 53, 32), u'uid': u'428524'}, u'timestamp': u'2011-03-28T15:53:32Z', u'pos': [53.4855118, -2.2370689], u'created_by': u'Potlatch 0.10f', u'name': u'Cafe North', u'version': u'2', u'user': u'Daveysawyer', u'_id': ObjectId('592ca2a7230fad747dcc8ce0'), u'type': u'node', u'id': u'357201545', u'description': u'Free wifi available for customers'}
{u'changeset': u'42000472', u'amenity': u'cafe', u'uid': u'3735060', u'created': {u'changeset': u'42000472', u'version': u'2', u'user': u'Gregory Walton', u'timestamp': datetime.datetime(2016, 9, 8, 10, 33, 26), u'uid': u'3735060'}, u'timestamp': u'2016-09-08T10:33:26Z', u'pos': [53.4752422, -2.2324674], u'version': u'2', u'user': u'Gregory Walton', u'_id': ObjectId('592ca2c2230fad747dddd58b'), u'type': u'node', u'id': u'3878522015', u'name': u'Cafe North'

### Findings from the above invistgation :
Only one Cafe which is Costa Cafe at Xford Street showing in the query resualt.Also Cafe North doenst have address information.This indicates that the address is not a mandatory field when the contributor enter a record.


## Conclusion and Imporvements:
Now we have done with cleaning parts of the data.It's importand to address the Issues of the data source and list some improvements. Considering OpenStreetMaps is an open source project and the Data entered by contributors.One of the issues I faced that there is duplication in the data which means we can find the same Cafe shope entered twice. my suggesstion for this case is that when the contributer point the location of the place there's a list nearby places appear so he knows that place is already there in the dataset.Moreover,The address or some attributes were missing or left emplty in a record/row. The solution for this case is that we enhance the validation of the text boxs.Adding restriction and validation rules will help us ensure both the accuracy ,and the correctness.

There're few potential issues may be raised when implementating this solution.One of which is limiting the contributers and they will spend more time in completing one record so this may decreese the willing of contributing and entring more data.Alsoe enhancing the level of validating and showing nearby location features would be a cost since a senior of engineers with good techincal knowlege is required. 

In conclusion, Applying the suggestion will be a cost by requireing a dedicated team for the implementation.Also wrangling data and produce more insights without improving the data quality will need more effort and time.I belive that we can explore that data in many different ways that will produce differents outcomes.